# Proyecto Final - Machine Learning 1
- Ricardo Andrés Cortés Coronell
- Sebastian Castellanos Estupiñan
- Andrés Felipe Sanchez Rincón


## Objetivo del Proyecto
Desarrollar un sistema predictivo multimodelo para la identificación y clasificación de patrones de consumo de varias sustancias mediante aprendizaje supervisado, comparando el desempeño mediante metricas de 3 algoritmos (Random Forest, XGBoost, SVM) bajo diferentes estrategias de balanceo de clases y selección de caracteristicas clave.

## Sobre el Dataset

**Fuente:** https://archive.ics.uci.edu/dataset/373/drug+consumption+quantified

Este dataset contiene información de 1885 individuos y 32 variables (13 de entrada y 19 objetivos), incluyendo:

Datos demográficos: edad, género, país, educación, etnia

Rasgos de personalidad (modelo de los cinco grandes): Nscore, Escore, Oscore, Ascore, Cscore

Otros factores psicológicos: impulsividad (BIS-11), búsqueda de sensaciones (ImpSS)

Clasificación del consumo de 19 tipos de sustancias, como alcohol, cannabis, cocaína, LSD, etc.
Cada droga tiene una etiqueta categórica con 7 posibles clases (CL0 a CL6) según la frecuencia de consumo.


## Diccionario del Dataset

### Variables de entrada

| Columna     | Descripción                                                                                       |
|-------------|---------------------------------------------------------------------------------------------------|
| `ID`        | Número identificador (sin relación con la persona real)                                           |
| `Age`       | Edad codificada:                                                                                  |
|             | - `-0.95197`: 18-24, `-0.07854`: 25-34, `0.49788`: 35-44, `1.09449`: 45-54, `1.82213`: 55-64, `2.59171`: 65+ |
| `Gender`    | Género codificado: `0.48246` = Mujer, `-0.48246` = Hombre                                          |
| `Education` | Nivel educativo:                                                                                   |
|             | - `-2.43591`: < 16 años, `-1.73790`: a los 16, `-1.43719`: a los 17, `-1.22751`: a los 18          |
|             | - `-0.61113`: Universidad sin título, `-0.05921`: Técnico, `0.45468`: Profesional, `1.16365`: Maestría, `1.98437`: Doctorado |
| `Country`   | País de residencia codificado:                                                                    |
|             | - `0.96082`: UK, `-0.57009`: USA, `0.24923`: Canadá, `-0.09765`: Australia, etc.                  |
| `Ethnicity` | Grupo étnico codificado: `-0.31685`: Blanco, `-0.50212`: Asiático, `-1.10702`: Negro, etc.         |
| `Nscore`    | Neuroticismo (NEO-FFI-R) normalizado                                                               |
| `Escore`    | Extraversión (NEO-FFI-R) normalizado                                                               |
| `Oscore`    | Apertura a nuevas experiencias (NEO-FFI-R)                                                         |
| `Ascore`    | Amabilidad (NEO-FFI-R)                                                                             |
| `Cscore`    | Responsabilidad (NEO-FFI-R)                                                                        |
| `Impulsive` | Impulsividad medida por BIS-11                                                                    |
| `SS`        | Sensación de búsqueda medida por ImpSS                                                             |

---

### Variables objetivo (por sustancia)

Cada sustancia tiene una columna con una de las siguientes clases (codificadas en el dataset como `CLx`):

| Código  | Interpretación             |
|---------|----------------------------|
| CL0     | Nunca ha usado             |
| CL1     | Usado hace más de 10 años  |
| CL2     | Usado en la última década  |
| CL3     | Usado en el último año     |
| CL4     | Usado en el último mes     |
| CL5     | Usado en la última semana  |
| CL6     | Usado en el último día     |

**Columnas de consumo:**

- `Alcohol`
- `Amphet` (Anfetaminas)
- `Amyl` (Nitrito de amilo)
- `Benzos` (Benzodiacepinas)
- `Caff` (Cafeína)
- `Cannabis` (Marihuana)
- `Chocolate`
- `Coke` (Cocaína)
- `Crack`
- `Ecstasy` (Éxtasis)
- `Heroin`
- `Ketamine`
- `Legalh` (Drogas legales)
- `LSD`
- `Meth` (Metadona)
- `Mushrooms` (Hongos)
- `Nicotine`
- `Semer` (Droga ficticia Semeron)
- `VSA` (Abuso de sustancias volátiles)

---

## Visualización del Dataset

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from prefect import flow, task

# Nombres de columnas explicados
column_names = [
    "ID", "Age", "Gender", "Education", "Country", "Ethnicity",
    "Nscore", "Escore", "Oscore", "Ascore", "Cscore",
    "Impulsive", "SS",
    "Alcohol", "Amphet", "Amyl", "Benzos", "Caff", "Cannabis", "Chocolate",
    "Coke", "Crack", "Ecstasy", "Heroin", "Ketamine", "Legalh",
    "LSD", "Meth", "Mushrooms", "Nicotine", "Semer", "VSA"
]

df = pd.read_csv("drug_consumption.csv", header=None, names=column_names)
df.head()


,ID,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,...,Ecstasy,Heroin,Ketamine,Legalh,LSD,Meth,Mushrooms,Nicotine,Semer,VSA
0,1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
1,2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,...,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
2,3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,...,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
3,4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,...,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
4,5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,...,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0
